In [ ]:
pip install eli5

     |████████████████████████████████| 106 kB 5.2 MB/s 


In [ ]:
#Data analysis
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
# from sklearn_crfsuite import CRF, scorers, metrics
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scipy.stats
import eli5

## LOAD DATASET

In [ ]:
df = pd.read_csv('/content/singgalang.csv', header=None)

In [ ]:
df.head()

,0
0,Ia\tO
1,menjabat\tO
2,sebagai\tO
3,Presiden\tO
4,ketiga\tO


In [ ]:
df_new = df[0].str.split('\t' , expand=True)

In [ ]:
df_new.head(10)

,0,1
0,Ia,O
1,menjabat,O
2,sebagai,O
3,Presiden,O
4,ketiga,O
5,Mesir,Place
6,pada,O
7,periode,O
8,15,O
9,Oktober,O


## EXPLORATORY DATA ANALYSIS

In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641099 entries, 0 to 641098
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       620953 non-null  object
 1   1       620920 non-null  object
dtypes: object(2)
memory usage: 9.8+ MB


In [ ]:
df_new.isnull()

,0,1
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
641094,False,False
641095,False,False
641096,False,False
641097,False,False


In [16]:
#add header to dataset
df_new = df_new.rename(columns={0 : 'words', 1 : 'tag'})
df_new

,words,tag
0,Ia,O
1,menjabat,O
2,sebagai,O
3,Presiden,O
4,ketiga,O
...,...,...
641094,dari,O
641095,jawa,O
641096,",",O
641097,mereka,O


## DATA PREPROCESSING

In [17]:
#Men-check distribusi Tag
df_new.groupby('tag').size().reset_index(name='counts')

,tag,counts
0,O,559536
1,Organisation,5125
2,Person,14278
3,Place,41981


###1. Case Folding

In [18]:
#Fungsi kita melakukan case folding adalah, agar setiap kata yg ada dalam dataset memiliki bentuk yg sama
df_new['words'] = df_new['words'].str.lower()

In [19]:
df_new.sample(10)

,words,tag
289797,",",O
434187,ilya,Person
141818,2005,O
135596,berklee,O
591313,sekarang,O
626549,trombol,Place
226668,yang,O
456787,menggunakan,O
54369,pendirian,O
542017,``,O


## FEATURE EXTRACTION

In [24]:
#Kita menggunakan Word2vec untuk mengekstraksi fitur
import nltk
import gensim
from gensim.models import Word2Vec
model = Word2Vec(sentences=df_new['words'], size=300, window=5, min_count=1, workers=4, sg=0)

TypeError: ignored